##### Solr info
---

# Zoeken met Solr

[Solr](http://lucene.apache.org/solr/) is de zoek-machine achter EASY. Naast de gebruikelijke zoek-interfaces op EASY kun je Solr-EASY ook via een eigen interface benaderen: de [Admin Page](http://easy01.dans.knaw.nl:8080/solr/datasets/admin/). Met de [Solr query syntax](https://wiki.apache.org/solr/SolrQuerySyntax) kun je geavanceerde queries samenstellen en de *Full interface* van de **Admin Page** geeft ook de mogelijkheid de output van het zoekresultaat te bepalen.

Wat we hier boven beschreven, kunnen we ook allemaal met de Solr API, de programmeerbare interface van Solr. De meerwaarde om dit via JupyterLab te doen bestaat onder andere uit:

- Je kunt je queries automatiseren
- Resultaat naar een Pandas DataFrame schrijven. Daarmee kun je
    - Nog meer en meer geavanceerde analyses op je zoekresultaten doen
    - Je zoekresultaat naar Excel wegschrijven
    - Linken naar datasets in je zoekresultaat.

## Solr aanroepen

De volgende methode accepteert een query, het zoekresultaat wordt weggeschreven naar een Pandas DataFrame. Merk op dat we het resultaat beperken tot de eerste 200 ('rows=200'). Pas dit eventueel aan aan wat je wil bereiken. We instrueren Solr ook om het resultaat terug te geven als csv (`wt=csv`). csv leest makkelijk in in een Pandas DataFrame.

In [1]:
import pandas as pd
import urllib

def search(query):
    url = 'http://easy01.dans.knaw.nl:8080/solr/datasets/select?wt=csv&rows=200&q=' + urllib.parse.quote(query)
    df = pd.read_csv(url)
    return df

### Alle zoekvelden

Om de werking van onze zoek-methode te demonstreeren èn om een overzicht te geven van de zoekvelden die de index easyDatasets gebruikt, voeren we een eenvoudige query uit, die maar één resultaat geeft, en draaien, voor de duidelijkheid, even kolommen en rijen van de resultaat DataFrame om (kolommen worden rijen). 

De query gebruikt het zoekveld `sid` om een enkel resultaat te krijgen. Achter de aanroep zetten we `T` voor [Transform](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.T.html). Kijk maar wat er gebeurt als je die `T` (en de punt) weg laat. Voor de duidelijkheid, we hadden het ook zo kunnen schrijven:
```
query = 'sid:"easy-dataset:47169"'
resultaat = search(query)
resultaat.T
```

In [2]:
query = 'sid:easy-dataset:114164'
search(query).T

,0
dc_coverage,"Provincie Groningen,Gemeente Groningen,Groning..."
emd_date_available_formatted,2018-11-27
amd_assignee_id,kbakker
dc_title_s,Archeologische begeleiding aanleg gasleiding O...
dc_source,NaN
type,"easy-dataset,dataset"
dc_description,De aanleiding tot het archeologisch onderzoek ...
emd_date_created_formatted,20181126-01-01T00:00:00.000+0100
dc_subject,VWAL
sid,easy-dataset:114164


Je ziet dat de lengte van de velden beperkt wordt. Wil je dat niet, dan kun je Pandas een andere instelling geven. Zet voor de query de regel

```
pd.options.display.max_colwidth = 0
```
... en kijk wat er gebeurt.

## Velden beperken, linken

Vooral als je meeredere zoekresultaten hebt en en je wil die onder elkaar afbeelden (zonder te ***T***ransformeren) is het handig om niet alle kolommen weer te geven. Kolommen kun je beperken in je query, maar ook door op te geven aan het Pandas DataFrame welke kolommen gerendered moeten worden. Nog een truc: linken naar de dataset in EASY.

In [3]:
# Maakt van een gegeven val een html-tag die linkt naar EASY
def link_easy(val):
    prefix = 'https://easy.dans.knaw.nl/ui/datasets/id/'
    return '<a target="_blank" href="{}{}">{}</a>'.format(prefix, val, val)

In [4]:
# Test link_easy
link_easy('dataset:123')

'<a target="_blank" href="https://easy.dans.knaw.nl/ui/datasets/id/dataset:123">dataset:123</a>'

In [5]:
# We willen alleen maar deze velden zien
cols = ['sid', 'ds_state', 'dc_title', 'emd_date_published']

# Geen beperking aan kolombreedte...
pd.options.display.max_colwidth = 0

query = 'dc_coverage:Groningen'
search(query).head(10)[cols].style.format({'sid': link_easy})

,sid,ds_state,dc_title,emd_date_published
0,easy-dataset:17056,PUBLISHED,"Opende,Steekproef 2006-07/20",2009-11-05T15:31:44Z
1,easy-dataset:17115,PUBLISHED,"Den Ham\, Veldstraat,Steekproef 2006-07/05",2009-11-05T15:43:12Z
2,easy-dataset:65743,PUBLISHED,Archeologisch onderzoek wijk 'Kostverloren' te Groningen,2016-08-23T06:36:44.073Z
3,easy-dataset:78083,PUBLISHED,"Archeologische begeleiding van het plaatsen van ondergrondse containers aan het Boterdiep te Groningen\, gemeente Groningen (GR)",2018-03-19T09:16:13.544Z
4,easy-dataset:30202,PUBLISHED,"Gemeente Grootegast\, terreinen Klokkenstoel en Premonstratenser klooster : archeologisch onderzoek,RAAPRAPPORT 323",2009-06-05T10:00:51Z
5,easy-dataset:45208,PUBLISHED,"Stadse Fratsen 1,Een Laat-Middeleeuws runderskelet uit de opgraving Groningen\, Agricolastraat 30-32",2011-01-27T09:44:08Z
6,easy-dataset:17768,PUBLISHED,Zandkoppen in de Oude Veenkoloniën. Een Archeologische Kwaliteitsbepaling,2009-11-05T20:26:14Z
7,easy-dataset:561,SUBMITTED,Compupaper [newspapers],nan
8,easy-dataset:18765,PUBLISHED,"Rapportage Aanvullende Archeologische Inventarisatie 1 Winschoten,SYNTHEGRA 2002 057",nan
9,easy-dataset:41062,PUBLISHED,"Getuigen Verhalen\, Ooggetuigen van het Scholtenhuis\, interview 05",nan


Wat gebeurde hier allemaal?

- De methode `link_easy` maakt van een gegeven waarde een link naar EASY. Dat zie je ook in de test daar onder.
- De `cols = ['sid', 'ds_state', 'dc_title', 'emd_date_published']` is Python om een [list](https://www.tutorialspoint.com/python/python_lists.htm) aan te maken. Een list is een basis data-structuur in Python.
- De query beperkt het zoekresultaat tot dc:coverage 'Groningen'.
- De search wordt uitgevoerd `search(query)`
- Het zoekresultaat wordt beperkt tot de eerste 10 resultaten `head(10)`
- De kolommen worden beperkt `[cols]`
- De waarde van het veld 'sid' wordt gerendered door de methode `link_easy` en afgebeeld als link `style.format({'sid': link_easy})`

### Het resultaat opslaan als csv of Excel bestand

We voeren dezelfde query weer uit maar slaan nu het resultaat op in een variabele: `df`.

In [6]:
query = 'dc_coverage:Groningen'
df = search(query)

We zien niks verschijnen, het resultaat is opgeslagen in variabele `df`. Hoe groot is onze tabel met zoekresultaten?

In [7]:
df.shape

(200, 42)

`shape` (zonder haakjes) geeft de dimensies van de dataframe: 200 rijen (we hadden het zoekresultaat beperkt tot 200) en 42 kolommen.

De methoden `to_csv` en `to_excel` van DataFrame slaan de dataframe op als bestand. Maak eerst een nieuwe directory aan in het linker deel van je Lab. Noem de directory 'data'.

Bewaar de dataframe integraal als csv:

In [8]:
df.to_csv('data/integraal.csv')

Sla de dataframe op als Excel, maar beperk de kolommen tot de geselecteerde:

In [9]:
df[cols].to_excel('data/selected.xls', index=None)

Met `index=None` voorkomen we dat Pandas een extra kolom met indexnummers naar Excel schrijft.

### Inlezen van bestanden

Inlezen van data gaat ook. Met `index_col=0` zeggen we tegen Pandas "de data heeft al een index-kolom en wel op plaats `0`". Anders maakt Pandas vrolijk weer een nieuwe index-kolom aan.

In [10]:
df2 = pd.read_csv('data/integraal.csv', index_col=0)
df2[['dc_identifier', 'emd_audience']].head()

,dc_identifier,emd_audience
0,"a02133,twips.dans.knaw.nl--966061926505295840-1257435104501,urn:nbn:nl:ui:13-rtl-87v,easy-dataset:17056,10.17026/dans-x4x-pdkp",easy-discipline:2
1,"a02127,twips.dans.knaw.nl--449206700126139811-1257435792724,urn:nbn:nl:ui:13-4js-3wy,easy-dataset:17115,10.17026/dans-2bs-sgk6",easy-discipline:2
2,"urn:nbn:nl:ui:13-fshe-vp,10.17026/dans-zxp-4fe8,64466,51152112,a27621,easy-dataset:65743",easy-discipline:2
3,"urn:nbn:nl:ui:13-f0-pr2c,10.17026/dans-zvx-6rqw,3994868100,1875-5313,a33936,Projectnummer 55135516,easy-dataset:78083",easy-discipline:2
4,"a06214,twips.dans.knaw.nl--4411347785204897798-1244196051210,urn:nbn:nl:ui:13-lbm-yvw,easy-dataset:30202,10.17026/dans-xyy-q73h",easy-discipline:2


Door het inperken van kolommen die worden afgebeeld (let op de dubbele `[[  ]]` om de rij met kolomnamen) en door het beperken van het aantal rijen (`head()` zonder parameter geeft standaard 5 rijen) krijgen we weer een ander gezicht op de data.